In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from fairlearn.preprocessing import CorrelationRemover
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.datasets import StandardDataset

# === Load MIMIC dataset ===
df = pd.read_csv("mimic_synthetic_data_3400_samples_DECAF.csv", low_memory=False)

# === Drop rows with missing values ===
df = df.dropna()

# === Create binary label: LOS >= 345600 seconds (4 days) → 1, else 0 ===
df["label"] = (df["los_seconds"] >= 345600).astype(int)

# === Define binary race column: 1 if WHITE (privileged), else 0 ===
df["race_binary"] = (df["race"] == "WHITE").astype(int)

# === One-hot encode all other categorical string columns ===
df = pd.get_dummies(df, drop_first=True)

# === Split ===
X = df.drop(columns=["los_seconds", "label"])
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_test = y_test.reset_index(drop=True)
race_test = X_test["race_binary"].reset_index(drop=True)

# === Model training ===
def train_models(X_train, y_train, sample_weight=None):
    models = {
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "SVM": SVC(probability=True),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
    }
    for name, model in models.items():
        model.fit(X_train, y_train, sample_weight=sample_weight)
    return models

# === Fairness Evaluation ===
def evaluate_fairness(models, X_test, y_test, race_test, title):
    print(f"\nFairness Metrics: {title}")
    print("--------------------------------------------------------------------------")
    print(f"{'Model':<20} {'ABROCA':<10} {'ERD':<10} {'TPRD':<10} {'Fairness':<10}")
    print("--------------------------------------------------------------------------")

    for name, model in models.items():
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

        group_priv = race_test == 1  # WHITE
        group_unpriv = race_test == 0  # non-WHITE

        err_priv = np.mean(y_pred[group_priv] != y_test[group_priv])
        err_unpriv = np.mean(y_pred[group_unpriv] != y_test[group_unpriv])
        erd = err_unpriv - err_priv

        def tpr(y_true, y_pred):
            tp = np.sum((y_true == 1) & (y_pred == 1))
            fn = np.sum((y_true == 1) & (y_pred == 0))
            return tp / (tp + fn) if (tp + fn) > 0 else 0

        tpr_priv = tpr(y_test[group_priv], y_pred[group_priv])
        tpr_unpriv = tpr(y_test[group_unpriv], y_pred[group_unpriv])
        tprd = tpr_unpriv - tpr_priv

        abroca = np.nan
        if y_prob is not None:
            try:
                fpr_priv, tpr_priv_vals, _ = roc_curve(y_test[group_priv], y_prob[group_priv])
                fpr_unpriv, tpr_unpriv_vals, _ = roc_curve(y_test[group_unpriv], y_prob[group_unpriv])
                auc_priv = auc(fpr_priv, tpr_priv_vals)
                auc_unpriv = auc(fpr_unpriv, tpr_unpriv_vals)
                abroca = abs(auc_priv - auc_unpriv)
            except:
                pass

        fairness = (3 - abs(abroca) - abs(erd) - abs(tprd)) / 3 if not np.isnan(abroca) else float("nan")
        print(f"{name:<20} {abroca:<10.4f} {erd:<10.4f} {tprd:<10.4f} {fairness:<10.4f}")

# === 0. Absolute Fairness ===
models_abs = train_models(X_train, y_train)
evaluate_fairness(models_abs, X_test, y_test, race_test, "Absolute (No Mitigation)")

# === 1. Suppression ===
X_train_sup = X_train.drop(columns=["race_binary"], errors="ignore")
X_test_sup = X_test.drop(columns=["race_binary"], errors="ignore")
models_sup = train_models(X_train_sup, y_train)
evaluate_fairness(models_sup, X_test_sup, y_test, race_test, "Suppression")

# === 2. Correlation Remover ===
cr = CorrelationRemover(sensitive_feature_ids=["race_binary"])
cr.fit(X_train)
X_train_cr = cr.transform(X_train)
X_test_cr = cr.transform(X_test)
models_cr = train_models(X_train_cr, y_train)
evaluate_fairness(models_cr, X_test_cr, y_test, race_test, "Correlation Remover")

# === 3. Disparate Impact Remover ===
df_train = pd.concat([X_train, y_train.rename("label")], axis=1)
dataset_dir = StandardDataset(df_train, label_name="label", favorable_classes=[0],
                               protected_attribute_names=["race_binary"], privileged_classes=[[1]])
dir = DisparateImpactRemover(repair_level=1.0)
transformed = dir.fit_transform(dataset_dir)
X_train_dir = pd.DataFrame(transformed.features, columns=X_train.columns)
models_dir = train_models(X_train_dir, y_train)
evaluate_fairness(models_dir, X_test, y_test, race_test, "Disparate Impact Remover")

# === 4. Reweighing ===
dataset_rw = StandardDataset(df_train, label_name="label", favorable_classes=[0],
                             protected_attribute_names=["race_binary"], privileged_classes=[[1]])
rw = Reweighing(privileged_groups=[{"race_binary": 1}], unprivileged_groups=[{"race_binary": 0}])
transformed_rw = rw.fit_transform(dataset_rw)
X_train_rw = pd.DataFrame(transformed_rw.features, columns=X_train.columns)
models_rw = train_models(X_train_rw, y_train, sample_weight=transformed_rw.instance_weights)
evaluate_fairness(models_rw, X_test, y_test, race_test, "Reweighing")


c:\Users\sshakibhamedan\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\sshakibhamedan\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
pip install 'aif360[inFairness]'



Fairness Metrics: Absolute (No Mitigation)
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0826     -0.0644    0.1547     0.8994    
Logistic Regression  0.0247     -0.0284    0.0311     0.9719    
Random Forest        0.0175     -0.0160    0.0617     0.9683    
SVM                  0.0077     -0.0744    0.0000     0.9726    
XGBoost              0.0141     -0.0411    0.1102     0.9449    

Fairness Metrics: Suppression
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0936     -0.0703    0.1827     0.8845    
Logistic Regression  0.0247     -0.0284    0.0311     0.9719    
Random Forest        0.0193     -0.0093 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from fairlearn.preprocessing import CorrelationRemover
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.datasets import StandardDataset

# === Load MIMIC dataset ===
df = pd.read_csv("generated_data_Our_prompts_MIMIC.csv", low_memory=False)

# === Drop rows with missing values ===
df = df.dropna()

# === Create binary label: LOS >= 345600 seconds (4 days) → 1, else 0 ===
df = df.rename(columns={"los_seconds": "label"})


# === Define binary race column: 1 if WHITE (privileged), else 0 ===
df["race_binary"] = (df["race"] == "WHITE").astype(int)

# === One-hot encode all other categorical string columns ===
df = pd.get_dummies(df, drop_first=True)

# === Split ===
X = df.drop(columns=[ "label", "stay_id", "subject_id", "hadm_id"])
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_test = y_test.reset_index(drop=True)
race_test = X_test["race_binary"].reset_index(drop=True)

# === Model training ===
def train_models(X_train, y_train, sample_weight=None):
    models = {
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "SVM": SVC(probability=True),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
    }
    for name, model in models.items():
        model.fit(X_train, y_train, sample_weight=sample_weight)
    return models

# === Fairness Evaluation ===
def evaluate_fairness(models, X_test, y_test, race_test, title):
    print(f"\nFairness Metrics: {title}")
    print("--------------------------------------------------------------------------")
    print(f"{'Model':<20} {'ABROCA':<10} {'ERD':<10} {'TPRD':<10} {'Fairness':<10}")
    print("--------------------------------------------------------------------------")

    for name, model in models.items():
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

        group_priv = race_test == 1  # WHITE
        group_unpriv = race_test == 0  # non-WHITE

        err_priv = np.mean(y_pred[group_priv] != y_test[group_priv])
        err_unpriv = np.mean(y_pred[group_unpriv] != y_test[group_unpriv])
        erd = err_unpriv - err_priv

        def tpr(y_true, y_pred):
            tp = np.sum((y_true == 1) & (y_pred == 1))
            fn = np.sum((y_true == 1) & (y_pred == 0))
            return tp / (tp + fn) if (tp + fn) > 0 else 0

        tpr_priv = tpr(y_test[group_priv], y_pred[group_priv])
        tpr_unpriv = tpr(y_test[group_unpriv], y_pred[group_unpriv])
        tprd = tpr_unpriv - tpr_priv

        abroca = np.nan
        if y_prob is not None:
            try:
                fpr_priv, tpr_priv_vals, _ = roc_curve(y_test[group_priv], y_prob[group_priv])
                fpr_unpriv, tpr_unpriv_vals, _ = roc_curve(y_test[group_unpriv], y_prob[group_unpriv])
                auc_priv = auc(fpr_priv, tpr_priv_vals)
                auc_unpriv = auc(fpr_unpriv, tpr_unpriv_vals)
                abroca = abs(auc_priv - auc_unpriv)
            except:
                pass

        fairness = (3 - abs(abroca) - abs(erd) - abs(tprd)) / 3 if not np.isnan(abroca) else float("nan")
        print(f"{name:<20} {abroca:<10.4f} {erd:<10.4f} {tprd:<10.4f} {fairness:<10.4f}")

# === 0. Absolute Fairness ===
models_abs = train_models(X_train, y_train)
evaluate_fairness(models_abs, X_test, y_test, race_test, "Absolute (No Mitigation)")

# === 1. Suppression ===
X_train_sup = X_train.drop(columns=["race_binary"], errors="ignore")
X_test_sup = X_test.drop(columns=["race_binary"], errors="ignore")
models_sup = train_models(X_train_sup, y_train)
evaluate_fairness(models_sup, X_test_sup, y_test, race_test, "Suppression")

# === 2. Correlation Remover ===
cr = CorrelationRemover(sensitive_feature_ids=["race_binary"])
cr.fit(X_train)
X_train_cr = cr.transform(X_train)
X_test_cr = cr.transform(X_test)
models_cr = train_models(X_train_cr, y_train)
evaluate_fairness(models_cr, X_test_cr, y_test, race_test, "Correlation Remover")

# === 3. Disparate Impact Remover ===
df_train = pd.concat([X_train, y_train.rename("label")], axis=1)
dataset_dir = StandardDataset(df_train, label_name="label", favorable_classes=[0],
                               protected_attribute_names=["race_binary"], privileged_classes=[[1]])
dir = DisparateImpactRemover(repair_level=1.0)
transformed = dir.fit_transform(dataset_dir)
X_train_dir = pd.DataFrame(transformed.features, columns=X_train.columns)
models_dir = train_models(X_train_dir, y_train)
evaluate_fairness(models_dir, X_test, y_test, race_test, "Disparate Impact Remover")

# === 4. Reweighing ===
dataset_rw = StandardDataset(df_train, label_name="label", favorable_classes=[0],
                             protected_attribute_names=["race_binary"], privileged_classes=[[1]])
rw = Reweighing(privileged_groups=[{"race_binary": 1}], unprivileged_groups=[{"race_binary": 0}])
transformed_rw = rw.fit_transform(dataset_rw)
X_train_rw = pd.DataFrame(transformed_rw.features, columns=X_train.columns)
models_rw = train_models(X_train_rw, y_train, sample_weight=transformed_rw.instance_weights)
evaluate_fairness(models_rw, X_test, y_test, race_test, "Reweighing")



Fairness Metrics: Absolute (No Mitigation)
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0293     -0.0015    -0.0130    0.9854    
Logistic Regression  0.0082     -0.0014    -0.0118    0.9929    
Random Forest        0.1209     0.0062     -0.0098    0.9544    
SVM                  0.0358     -0.0069    0.0000     0.9858    
XGBoost              0.0799     0.0052     -0.0158    0.9663    

Fairness Metrics: Suppression
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0546     -0.0010    -0.0221    0.9741    
Logistic Regression  0.0081     -0.0014    -0.0118    0.9929    
Random Forest        0.1010     0.0101  

C:\Users\sshakibhamedan\AppData\Roaming\Python\Python39\site-packages\aif360\datasets\standard_dataset.py:143: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[pos, label_name] = favorable_label



Fairness Metrics: Disparate Impact Remover
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0688     -0.0111    -0.0132    0.9690    
Logistic Regression  0.0060     -0.0033    -0.0118    0.9930    
Random Forest        0.0767     0.0118     -0.0139    0.9659    
SVM                  0.0139     -0.0069    0.0000     0.9931    
XGBoost              0.0529     -0.0002    -0.0073    0.9799    


C:\Users\sshakibhamedan\AppData\Roaming\Python\Python39\site-packages\aif360\datasets\standard_dataset.py:143: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[pos, label_name] = favorable_label



Fairness Metrics: Reweighing
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0201     0.0146     -0.0313    0.9780    
Logistic Regression  0.0080     0.0025     -0.0118    0.9926    
Random Forest        0.1173     0.0021     -0.0119    0.9562    
SVM                  0.0099     -0.0069    0.0000     0.9944    
XGBoost              0.0583     0.0016     -0.0208    0.9731    


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from fairlearn.preprocessing import CorrelationRemover
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.datasets import StandardDataset

# === Load MIMIC dataset ===
df = pd.read_csv("generated_data_CLLM_prompt_Mimic.csv", low_memory=False)

# === Drop rows with missing values ===
df = df.dropna()

# === Create binary label: LOS >= 345600 seconds (4 days) → 1, else 0 ===
df = df.rename(columns={"los_seconds": "label"})


# === Define binary race column: 1 if WHITE (privileged), else 0 ===
df["race_binary"] = (df["race"] == "WHITE").astype(int)

# === One-hot encode all other categorical string columns ===
df = pd.get_dummies(df, drop_first=True)

# === Split ===
X = df.drop(columns=[ "label", "stay_id", "subject_id", "hadm_id"])
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_test = y_test.reset_index(drop=True)
race_test = X_test["race_binary"].reset_index(drop=True)

# === Model training ===
def train_models(X_train, y_train, sample_weight=None):
    models = {
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "SVM": SVC(probability=True),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
    }
    for name, model in models.items():
        model.fit(X_train, y_train, sample_weight=sample_weight)
    return models

# === Fairness Evaluation ===
def evaluate_fairness(models, X_test, y_test, race_test, title):
    print(f"\nFairness Metrics: {title}")
    print("--------------------------------------------------------------------------")
    print(f"{'Model':<20} {'ABROCA':<10} {'ERD':<10} {'TPRD':<10} {'Fairness':<10}")
    print("--------------------------------------------------------------------------")

    for name, model in models.items():
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

        group_priv = race_test == 1  # WHITE
        group_unpriv = race_test == 0  # non-WHITE

        err_priv = np.mean(y_pred[group_priv] != y_test[group_priv])
        err_unpriv = np.mean(y_pred[group_unpriv] != y_test[group_unpriv])
        erd = err_unpriv - err_priv

        def tpr(y_true, y_pred):
            tp = np.sum((y_true == 1) & (y_pred == 1))
            fn = np.sum((y_true == 1) & (y_pred == 0))
            return tp / (tp + fn) if (tp + fn) > 0 else 0

        tpr_priv = tpr(y_test[group_priv], y_pred[group_priv])
        tpr_unpriv = tpr(y_test[group_unpriv], y_pred[group_unpriv])
        tprd = tpr_unpriv - tpr_priv

        abroca = np.nan
        if y_prob is not None:
            try:
                fpr_priv, tpr_priv_vals, _ = roc_curve(y_test[group_priv], y_prob[group_priv])
                fpr_unpriv, tpr_unpriv_vals, _ = roc_curve(y_test[group_unpriv], y_prob[group_unpriv])
                auc_priv = auc(fpr_priv, tpr_priv_vals)
                auc_unpriv = auc(fpr_unpriv, tpr_unpriv_vals)
                abroca = abs(auc_priv - auc_unpriv)
            except:
                pass

        fairness = (3 - abs(abroca) - abs(erd) - abs(tprd)) / 3 if not np.isnan(abroca) else float("nan")
        print(f"{name:<20} {abroca:<10.4f} {erd:<10.4f} {tprd:<10.4f} {fairness:<10.4f}")

# === 0. Absolute Fairness ===
models_abs = train_models(X_train, y_train)
evaluate_fairness(models_abs, X_test, y_test, race_test, "Absolute (No Mitigation)")

# === 1. Suppression ===
X_train_sup = X_train.drop(columns=["race_binary"], errors="ignore")
X_test_sup = X_test.drop(columns=["race_binary"], errors="ignore")
models_sup = train_models(X_train_sup, y_train)
evaluate_fairness(models_sup, X_test_sup, y_test, race_test, "Suppression")

# === 2. Correlation Remover ===
cr = CorrelationRemover(sensitive_feature_ids=["race_binary"])
cr.fit(X_train)
X_train_cr = cr.transform(X_train)
X_test_cr = cr.transform(X_test)
models_cr = train_models(X_train_cr, y_train)
evaluate_fairness(models_cr, X_test_cr, y_test, race_test, "Correlation Remover")

# === 3. Disparate Impact Remover ===
df_train = pd.concat([X_train, y_train.rename("label")], axis=1)
dataset_dir = StandardDataset(df_train, label_name="label", favorable_classes=[0],
                               protected_attribute_names=["race_binary"], privileged_classes=[[1]])
dir = DisparateImpactRemover(repair_level=1.0)
transformed = dir.fit_transform(dataset_dir)
X_train_dir = pd.DataFrame(transformed.features, columns=X_train.columns)
models_dir = train_models(X_train_dir, y_train)
evaluate_fairness(models_dir, X_test, y_test, race_test, "Disparate Impact Remover")

# === 4. Reweighing ===
dataset_rw = StandardDataset(df_train, label_name="label", favorable_classes=[0],
                             protected_attribute_names=["race_binary"], privileged_classes=[[1]])
rw = Reweighing(privileged_groups=[{"race_binary": 1}], unprivileged_groups=[{"race_binary": 0}])
transformed_rw = rw.fit_transform(dataset_rw)
X_train_rw = pd.DataFrame(transformed_rw.features, columns=X_train.columns)
models_rw = train_models(X_train_rw, y_train, sample_weight=transformed_rw.instance_weights)
evaluate_fairness(models_rw, X_test, y_test, race_test, "Reweighing")



Fairness Metrics: Absolute (No Mitigation)
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0067     -0.0122    0.0103     0.9903    
Logistic Regression  0.0146     -0.0093    -0.0111    0.9883    
Random Forest        0.0282     0.0018     -0.0187    0.9838    
SVM                  0.0211     -0.0190    -0.0104    0.9832    
XGBoost              0.0005     -0.0108    -0.0182    0.9902    

Fairness Metrics: Suppression
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0424     -0.0471    0.0327     0.9593    
Logistic Regression  0.0144     -0.0093    -0.0111    0.9884    
Random Forest        0.0360     0.0181  

C:\Users\sshakibhamedan\AppData\Roaming\Python\Python39\site-packages\aif360\datasets\standard_dataset.py:143: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[pos, label_name] = favorable_label



Fairness Metrics: Disparate Impact Remover
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0136     -0.0225    0.0219     0.9807    
Logistic Regression  0.0150     0.0004     -0.0311    0.9845    
Random Forest        0.0006     -0.0057    -0.0376    0.9854    
SVM                  0.0153     -0.0170    -0.0139    0.9846    
XGBoost              0.0193     -0.0008    -0.0217    0.9861    


C:\Users\sshakibhamedan\AppData\Roaming\Python\Python39\site-packages\aif360\datasets\standard_dataset.py:143: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[pos, label_name] = favorable_label



Fairness Metrics: Reweighing
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0008     -0.0103    0.0227     0.9887    
Logistic Regression  0.0146     -0.0131    0.0085     0.9879    
Random Forest        0.0499     0.0156     -0.0308    0.9679    
SVM                  0.0211     -0.0171    -0.0137    0.9827    
XGBoost              0.0211     -0.0051    -0.0053    0.9895    


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from xgboost import XGBClassifier
from fairlearn.preprocessing import CorrelationRemover
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.datasets import StandardDataset

# === Load only real dataset ===
df = pd.read_csv("Real_MIMIC.csv", low_memory=False)

# === Drop missing values
df = df.dropna()

# === Binarize los_seconds and race
df["label"] = (df["los_seconds"] >= 345600).astype(int)
df["race_binary"] = (df["race"] == "WHITE").astype(int)

# === One-hot encode categorical variables
df = pd.get_dummies(df, drop_first=True)

# === Split into X, y
X = df.drop(columns=["los_seconds", "label"])
y = df["label"]

# === Train-test split (only real data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
race_test = X_test["race_binary"].reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# === XGBoost model training
def train_model(X_train, y_train, sample_weight=None):
    model = XGBClassifier(eval_metric="logloss", random_state=42)
    model.fit(X_train, y_train, sample_weight=sample_weight)
    return model

# === Fairness evaluation function
def evaluate_fairness(model, X_test, y_test, race_test, title):
    print(f"\nFairness Metrics: {title}")
    print("--------------------------------------------------------------------------")
    print(f"{'Model':<20} {'ABROCA':<10} {'ERD':<10} {'TPRD':<10} {'Fairness':<10}")
    print("--------------------------------------------------------------------------")

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    group_priv = race_test == 1
    group_unpriv = race_test == 0

    err_priv = np.mean(y_pred[group_priv] != y_test[group_priv])
    err_unpriv = np.mean(y_pred[group_unpriv] != y_test[group_unpriv])
    erd = err_unpriv - err_priv

    def tpr(y_true, y_pred):
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))
        return tp / (tp + fn) if (tp + fn) > 0 else 0

    tpr_priv = tpr(y_test[group_priv], y_pred[group_priv])
    tpr_unpriv = tpr(y_test[group_unpriv], y_pred[group_unpriv])
    tprd = tpr_unpriv - tpr_priv

    abroca = np.nan
    try:
        fpr_priv, tpr_priv_vals, _ = roc_curve(y_test[group_priv], y_prob[group_priv])
        fpr_unpriv, tpr_unpriv_vals, _ = roc_curve(y_test[group_unpriv], y_prob[group_unpriv])
        auc_priv = auc(fpr_priv, tpr_priv_vals)
        auc_unpriv = auc(fpr_unpriv, tpr_unpriv_vals)
        abroca = abs(auc_priv - auc_unpriv)
    except:
        pass

    fairness = (3 - abs(abroca) - abs(erd) - abs(tprd)) / 3 if not np.isnan(abroca) else float("nan")
    print(f"{'XGBoost':<20} {abroca:<10.4f} {erd:<10.4f} {tprd:<10.4f} {fairness:<10.4f}")

# === 0. Absolute (No Mitigation)
model_abs = train_model(X_train, y_train)
evaluate_fairness(model_abs, X_test, y_test, race_test, "Absolute (No Mitigation)")

# === 1. Suppression
X_train_sup = X_train.drop(columns=["race_binary"], errors="ignore")
X_test_sup = X_test.drop(columns=["race_binary"], errors="ignore")
model_sup = train_model(X_train_sup, y_train)
evaluate_fairness(model_sup, X_test_sup, y_test, race_test, "Suppression")

# === 2. Correlation Remover
cr = CorrelationRemover(sensitive_feature_ids=["race_binary"])
cr.fit(X_train)
X_train_cr = cr.transform(X_train)
X_test_cr = cr.transform(X_test)
model_cr = train_model(X_train_cr, y_train)
evaluate_fairness(model_cr, X_test_cr, y_test, race_test, "Correlation Remover")

# === 3. Disparate Impact Remover
df_train = pd.concat([X_train, y_train.rename("label")], axis=1)
dataset_dir = StandardDataset(df_train, label_name="label", favorable_classes=[0],
                              protected_attribute_names=["race_binary"], privileged_classes=[[1]])
dir = DisparateImpactRemover(repair_level=1.0)
transformed = dir.fit_transform(dataset_dir)
X_train_dir = pd.DataFrame(transformed.features, columns=X_train.columns)
model_dir = train_model(X_train_dir, y_train)
evaluate_fairness(model_dir, X_test, y_test, race_test, "Disparate Impact Remover")

# === 4. Reweighing
dataset_rw = StandardDataset(df_train, label_name="label", favorable_classes=[0],
                             protected_attribute_names=["race_binary"], privileged_classes=[[1]])
rw = Reweighing(privileged_groups=[{"race_binary": 1}], unprivileged_groups=[{"race_binary": 0}])
transformed_rw = rw.fit_transform(dataset_rw)
X_train_rw = pd.DataFrame(transformed_rw.features, columns=X_train.columns)
model_rw = train_model(X_train_rw, y_train, sample_weight=transformed_rw.instance_weights)
evaluate_fairness(model_rw, X_test, y_test, race_test, "Reweighing")


c:\Users\sshakibhamedan\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\sshakibhamedan\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
pip install 'aif360[inFairness]'



Fairness Metrics: Absolute (No Mitigation)
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
XGBoost              0.0221     0.0287     -0.0086    0.9802    

Fairness Metrics: Suppression
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
XGBoost              0.0221     0.0287     -0.0086    0.9802    

Fairness Metrics: Correlation Remover
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
XGBoost              0.0216     0.0218     -0.0151    0.9805    

Fairness Metrics: Disparate Impact Remover
--